In [6]:


import asyncio
from pyppeteer import launch
from pyquery import PyQuery as pq
import nest_asyncio
from bs4 import BeautifulSoup
nest_asyncio.apply()

 
async def main():
    
    href_list = []
    browser = await launch()
    page = await browser.newPage()
    for i in range(1, 10):

        myurl = 'https://sousuo.www.gov.cn/sousuo/search.shtml?code=17da70961a7&searchWord=%E8%83%BD%E6%BA%90%E5%AE%89%E5%85%A8&dataTypeId=107&sign=cbf6e6b5-f1a4-4b60-f71e-1109e025ced5&searchBy=all&pageNo={}'.format(str(i))
        url = myurl
        res = await page.goto(url=url)
        # 等待
        await asyncio.sleep(2)
        # print(await page.content())
        content = await page.content()

        soup = BeautifulSoup(content,'html.parser')
        
        # 找到所有的链接
        links = soup.find_all("a")
        # 提取链接的href属性
        href_list += [link.get("href") for link in links if link.get("href") and (link.get("href").startswith("https://") or link.get("href").startswith("https://"))]
    
    await browser.close()
    
    return href_list
 
href_list = asyncio.get_event_loop().run_until_complete(main())

In [7]:
len(href_list)

38

In [8]:
import pandas as pd
from tqdm import tqdm
async def main(href_list = href_list):
    
    browser = await launch()
    page = await browser.newPage()
    
    title_list = []
    time_list = []
    contexts_list = []
    department = []
    
    for url in tqdm(href_list):
    
        content_list = []
        res = await page.goto(url=url)
        # 等待
        await asyncio.sleep(1)
        # print(await page.content())
        content = await page.content()
        soup = BeautifulSoup(content,'html.parser')
        # print(soup)
        
        title = soup.find_all('title')[0].text.split('_')[0]
        time = soup.find_all('meta', attrs={'name':'firstpublishedtime'})[0].get('content')
        
        content_p = soup.find_all('div', attrs={'class':'pages_content'})[0].find_all('p')
        # print(content_p[10].text)
        for content in content_p:
            content_list.append(content.text)
        context = ''.join(content_list)
        
        # 匹配部门
        department_list = []
        with open('query_words.txt', 'r') as f:
            d_list = f.readlines()
        for d in d_list:
            if d.strip() in title+context:
                department_list.append(d.strip())
        
        
        
        title_list.append(title)
        time_list.append(time)
        contexts_list.append(context)
        department.append(','.join(department_list))
        
        # title = soup.select('title')[0].title
        # time = soup.select('meta[name="firstpublishedtime"]')[0].content
        
        # tim = soup.select('.font')[0].texts
        # time_1 = time.replace(tim,'')[:11]
        # contents = soup.select('p')
        # cont = []
        # for c in contents:
        #     con = c.text
        #     cont.append(con)
        # s = "".join(cont)
        # print(title)
        # print(time)
        # print(content)
    data = {'标题':title_list, '时间':time_list, '合作部门':department, '正文':contexts_list}
    data_df = pd.DataFrame(data)
    
    await browser.close()
    return data_df
    
data_df = asyncio.get_event_loop().run_until_complete(main())

100%|██████████| 38/38 [00:54<00:00,  1.43s/it]


In [9]:
data_df

,标题,时间,合作部门,正文
0,国家发展改革委 国家能源局 国家乡村振兴局关于实施农村电网巩固提升工程的指导意见,2023-08-10-15:00:00,"中央,内蒙,新疆,国务院,国家能源局,国家电网",国家发展改革委 国家能源局 国家乡村振兴局关于实施农村电网巩固提升工程的指导意见发改能源规〔...
1,启动十年 中巴经济走廊释放多重发展红利,2023-07-31-20:57:00,重庆,新华社伊斯兰堡7月31日电 （国际观察）启动十年 中巴经济走廊释放多重发展红利新华社记者 蒋...
2,习近平：加强基础研究 实现高水平科技自立自强,2023-07-31-15:24:00,"中央,北京,江苏,广东,苏州",加强基础研究 实现高水平科技自立自强※习近平今天，中央政治局进行第三次集体学习，内容是加强基...
3,中华人民共和国和圭亚那合作共和国联合声明（全文）,2023-07-31-09:43:00,"北京,上海,国务院,成都",新华社北京7月31日电中华人民共和国和圭亚那合作共和国联合声明（2023年7月31日）一、圭...
4,李强会见圭亚那总统阿里,2023-07-30-21:35:00,"北京,国务院",新华社北京7月30日电 国务院总理李强7月30日下午在人民大会堂会见来华出席第31届世界大学...
5,国家发展改革委联合多部门发文 推动现代煤化工产业健康发展,2023-07-30-07:44:00,国务院,国家发展改革委等部门发文推动现代煤化工产业健康发展为进一步强化煤炭主体能源地位，按照严控增量...
6,习近平会见圭亚那总统阿里,2023-07-28-12:31:00,"成都,重庆",新华社成都7月28日电（记者 袁帅、周相吉）7月28日上午，国家主席习近平在成都会见来华出席...
7,西部陆海新通道运行量质齐升,2023-07-28-10:57:00,"贵州,云南,陕西,青海,广西,新疆,重庆,昆明,贵阳,海口,毕节",从西部陆海新通道运输协调委员会获悉，截至7月20日，西部陆海新通道铁海联运班列今年已运输货物...
8,十四届全国人大常委会关于国有资产管理情况监督工作的五年规划（2023－2027）,2023-07-28-08:17:00,"中央,北京,自然资源部,国务院",新华社北京7月27日电十四届全国人大常委会关于国有资产管理情况监督工作的五年规划（2023－...
9,关于推动现代煤化工产业健康发展的通知,2023-07-27-21:06:00,"中央,新疆,水利部,环境部,生态环境部,自然资源部,国务院,国家能源局",国家发展改革委等部门关于推动现代煤化工产业健康发展的通知发改产业〔2023〕773号各省、自...


In [10]:
data_df.to_csv('data.csv', index=False)